## Importing the libraries

In [55]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
from torchvision import transforms
from torchvision.utils import save_image
import torch.optim as optim

## Download the Dataset

In [56]:
tranform_train = transforms.Compose([transforms.Resize((227,227)), transforms.RandomHorizontalFlip(p=0.7), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
tranform_test = transforms.Compose([transforms.Resize((227,227)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

torch.manual_seed(43)
train_ds = CIFAR10("data/", train=True, download=True, transform=tranform_train) 
val_size = 10000
train_size = len(train_ds) - val_size
train_ds, val_ds = random_split(train_ds, [train_size, val_size]) 
test_ds = CIFAR10("data/", train=False, download=True, transform=tranform_test) 

train_set = DataLoader(train_ds, batch_size=64, shuffle=True)
val_set = DataLoader(val_ds, batch_size=64, shuffle=False)
test_set = DataLoader(test_ds, batch_size=64, shuffle=False)

## Define the CNN model with different activation function on which  we want to test the data set on

### Model with Relu as activation function

In [57]:
class model_1(nn.Module):
    def __init__(self):
        super(model_1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 9216, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 10)


    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

### Model with tanh as Activation function

In [58]:
class model_2(nn.Module):
    def __init__(self):
        super(model_2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 9216, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 10)


    def forward(self,x):
        x = F.tanh(self.conv1(x))
        x = self.maxpool(x)
        x = F.tanh(self.conv2(x))
        x = self.maxpool(x)
        x = F.tanh(self.conv3(x))
        x = F.tanh(self.conv4(x))
        x = F.tanh(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        return x

### Model with sigmoid as activation function

In [59]:
class model_3(nn.Module):
    def __init__(self):
        super(model_3, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 9216, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 10)


    def forward(self,x):
        x = F.sigmoid(self.conv1(x))
        x = self.maxpool(x)
        x = F.sigmoid(self.conv2(x))
        x = self.maxpool(x)
        x = F.sigmoid(self.conv3(x))
        x = F.sigmoid(self.conv4(x))
        x = F.sigmoid(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

### Function to run the model

In [60]:
def runModel(model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
    epoch_loss = []
    model = model.to(device=device)
    learning_rate = 0.01
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr= learning_rate,momentum = 0.85)

    for epoch in range(15): 
        loss_ep = 0
    
        for batch_idx, (data, targets) in enumerate(train_set):
            data = data.to(device=device)
            targets = targets.to(device=device)
            ## Forward Pass
            optimizer.zero_grad()
            scores = model(data)
            loss = criterion(scores,targets)
            loss.backward()
            optimizer.step()
            loss_ep += loss.item()

        epoch_loss.append(loss_ep/len(train_set))
        print(f"Loss in epoch {epoch} is {loss_ep/len(train_set)}")

        with torch.no_grad():
            num_correct = 0
            num_samples = 0
            for batch_idx, (data,targets) in enumerate(val_set):
                data = data.to(device=device)
                targets = targets.to(device=device)
                ## Forward Pass
                scores = model(data)
                _, predictions = scores.max(1)
                num_correct += (predictions == targets).sum()
                num_samples += predictions.size(0)
            print(
                f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}"

            )
    plt.plot(epoch_loss)

### Test the model

In [61]:
def testModel(model):
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    num_correct = 0
    num_samples = 0
    for batch_idx, (data,targets) in enumerate(test_set):
        data = data.to(device="cuda")
        targets = targets.to(device="cuda")
        ## Forward Pass
        scores = model(data)
        _, predictions = scores.max(1)
        for target, prediction in zip(targets,predictions):
            if(target==prediction):
                correct_pred[classes[target]]+=1
            total_pred[classes[target]]+=1
        num_correct += (predictions == targets).sum()
        num_samples += predictions.size(0)
    print(
        f"Accuracy =  {float(num_correct) / float(num_samples) * 100:.2f}"
    )


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class  {classname:5s} = {accuracy:.1f} %')

In [62]:
relu_model = model_1()
runModel(relu_model)
testModel(relu_model)

In [63]:
tanh_model = model_1()
runModel(tanh_model)
test_model(tanh_model)

In [ ]:
sigmoid_model = model_1()
runModel(sigmoid_model)
test_model(sigmoid_model)